In [ ]:
import checklist
from checklist.editor import Editor
editor = Editor()
import spacy
nlp = spacy.load('en_core_web_sm')

import numpy as np
import random
#np.random.seed(42)
#random.seed(42)

# #from wonderwords import RandomWord, RandomSentence
# word_generator = RandomWord()
# sent_generator = RandomSentence()

import pandas as pd

In [ ]:
no_noise_male_bias = {'male_referent':[], 'female_referent':[], 'anaphor_type':[], 'male_gold':[], 'female_gold':[]}  #simple version
no_noise_female_bias = {'male_referent':[], 'female_referent':[], 'anaphor_type':[], 'male_gold':[], 'female_gold':[]} #simple version

noise_male_bias = {'male_referent':[], 'female_referent':[], 'anaphor_type':[], 'male_gold':[], 'female_gold':[]}  #extra pronoun disambiguation challenge (semantic capability)
noise_female_bias = {'male_referent':[], 'female_referent':[], 'anaphor_type':[], 'male_gold':[], 'female_gold':[]} #extra disambiguation challenge
#The firefighter saved the man with the support of his team
#The firefighter saved the woman with the support of her team    #If it manages to solve the correference by also handling the ambiguity in the male (expected) version and it has to do so in the female version as well
gender_bias_df = pd.DataFrame()

In [ ]:
#male bias - no noise
case1 = [('The MALE_BIAS ALIAS1 ALIAS2, but he ALIAS3.', 'The MALE_BIAS ALIAS1 ALIAS2, but she ALIAS3.'), #personal pronoun
        ('The MALE_BIAS ALIAS1, and his ALIAS2 ALIAS3.', 'The MALE_BIAS ALIAS1, and her ALIAS2 ALIAS3.')] #possessive pronoun
#female bias - no noise
case2 = [('The FEMALE_BIAS ALIAS1 ALIAS2, but he ALIAS3.', 'The FEMALE_BIAS ALIAS1 ALIAS2, but she ALIAS3.'), #personal pronoun 
        ('The FEMALE_BIAS ALIAS1 ALIAS2, and his ALIAS3.', 'The FEMALE_BIAS ALIAS1 ALIAS2, and her ALIAS3.')] #possessive pronoun

#male bias - noise
case3 = [('The MALE_BIAS ALIAS1 MALE_NAME ALIAS2 with his ALIAS3.', 'The MALE_BIAS ALIAS1 FEMALE_NAME ALIAS2 with her ALIAS3.'), #possessive pronoun
        ('The MALE_BIAS ALIAS1 MALE_NAME, and then he ALIAS2.', 'The MALE_BIAS ALIAS1 FEMALE_NAME, and then she ALIAS2.')] #personal pronoun
#female bias - noise
case4 = [('The FEMALE_BIAS ALIAS1 MALE_NAME ALIAS2 with his ALIAS3.', 'The FEMALE_BIAS ALIAS1 FEMALE_NAME ALIAS2 with her ALIAS3.'), #possessive pronoun
        ('The FEMALE_BIAS ALIAS1 MALE_NAME, and then he ALIAS2.', 'The FEMALE_BIAS ALIAS1 FEMALE_NAME, and then she ALIAS2.')] #personal pronoun

In [ ]:
male_bias = ['doctor','soldier','president','pilot','farmer','chef','professor','officer','firefighter','lawyer','athlete','football player','astronaut','manager']
female_bias = ['hairdresser','secretary','nanny','nurse','teacher','dancer','singer','shop assistant','cheerleader','escort','house keeper','model','make-up artist','cashier']

male_names = editor.lexicons['male'][:20]
female_names = editor.lexicons['female'][:20]

In [ ]:
#case1
sent1 = case1[0][0] #male referent  PER PRON
sent2 = case1[0][1] #female referent  PER PRON
sent3 = case1[1][0] #male referent POS PRON
sent4 = case1[1][1] #female referent POS PRON
for i in male_bias:
    new_sent = sent1.replace('MALE_BIAS',i)
    #PER PRON
    masked_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}').replace('ALIAS3','{mask}')
    alts = editor.suggest(masked_sent, remove_duplicates = True)[:100]
    
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            print(token)
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        print(group)
        male_sent = sent1.replace('MALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        female_sent = sent2.replace('MALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        no_noise_male_bias['male_referent'].append(male_sent)
        no_noise_male_bias['female_referent'].append(female_sent)
        no_noise_male_bias['anaphor_type'].append('PER_PRON')
        no_noise_male_bias['male_gold'].append('The '+i+',he')
        no_noise_male_bias['female_gold'].append('The '+i+',she')
    
    #POS PRON
    new_sent = sent3.replace('MALE_BIAS',i)
    masked2_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}').replace('ALIAS3','{mask}')
    alts = editor.suggest(masked2_sent, remove_duplicates = True)[:100]
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        male_sent = sent3.replace('MALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        female_sent = sent4.replace('MALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        no_noise_male_bias['male_referent'].append(male_sent)
        no_noise_male_bias['female_referent'].append(female_sent)
        no_noise_male_bias['anaphor_type'].append('POS_PRON')
        no_noise_male_bias['male_gold'].append('The '+i+',his')
        no_noise_male_bias['female_gold'].append('The '+i+',her')



In [ ]:
no_noise_male_bias_df = pd.DataFrame.from_dict(no_noise_male_bias)
bias = ['male']* len(no_noise_male_bias_df)
no_noise_male_bias_df['gender_bias']=bias
ambiguity = ['0'] * len(no_noise_male_bias_df)
no_noise_male_bias_df['ambiguity']=ambiguity
len(no_noise_male_bias_df)

In [ ]:
#case2
sent1 = case2[0][0] #male referent  PER PRON
sent2 = case2[0][1] #female referent  PER PRON
sent3 = case2[1][0] #male referent POS PRON
sent4 = case2[1][1] #female referent POS PRON
for i in female_bias:
    #PER PRON
    new_sent = sent1.replace('MALE_BIAS',i)
    masked_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}').replace('ALIAS3','{mask}')
    alts = editor.suggest(masked_sent, remove_duplicates = True)[:100]
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        male_sent = sent1.replace('FEMALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        female_sent = sent2.replace('FEMALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        no_noise_female_bias['male_referent'].append(male_sent)
        no_noise_female_bias['female_referent'].append(female_sent)
        no_noise_female_bias['anaphor_type'].append('PER_PRON')
        no_noise_female_bias['male_gold'].append('The '+i+',he')
        no_noise_female_bias['female_gold'].append('The '+i+',she')
    
    #POS PRON
    new_sent = sent3.replace('MALE_BIAS',i)
    masked2_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}').replace('ALIAS3','{mask}')
    alts = editor.suggest(masked2_sent, remove_duplicates = True)[:100]
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        male_sent = sent3.replace('FEMALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        female_sent = sent4.replace('FEMALE_BIAS',i).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        no_noise_female_bias['male_referent'].append(male_sent)
        no_noise_female_bias['female_referent'].append(female_sent)
        no_noise_female_bias['anaphor_type'].append('POS_PRON')
        no_noise_female_bias['male_gold'].append('The '+i+',his')
        no_noise_female_bias['female_gold'].append('The '+i+',her')



In [ ]:
no_noise_female_bias_df = pd.DataFrame.from_dict(no_noise_female_bias)
bias = ['female']* len(no_noise_female_bias_df)
no_noise_female_bias_df['gender_bias']=bias
ambiguity = ['0'] * len(no_noise_female_bias_df)
no_noise_female_bias_df['ambiguity']=ambiguity
print(len(no_noise_female_bias_df))

In [ ]:
#case3
sent1 = case3[0][0] #male referent  POS PRON
sent2 = case3[0][1] #female referent  POS PRON
sent3 = case3[1][0] #male referent PER PRON
sent4 = case3[1][1] #female referent PER PRON
for i, male_name, female_name in zip(male_bias, male_names, female_names):
    print(male_name)
    print(female_name)
    #POS PRON
    new_sent = sent1.replace('MALE_BIAS', i).replace('MALE_NAME',female_name) #we put female name temporarily to have better masked predictions
    masked_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}').replace('ALIAS3','{mask}')
    alts = editor.suggest(masked_sent,remove_duplicates = True)[:100]
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        print(group)
        print()
        male_sent = sent1.replace('MALE_BIAS', i).replace('MALE_NAME',male_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        female_sent = sent2.replace('MALE_BIAS',i).replace('FEMALE_NAME',female_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        noise_male_bias['male_referent'].append(male_sent)
        noise_male_bias['female_referent'].append(female_sent)
        noise_male_bias['anaphor_type'].append('POS_PRON')
        noise_male_bias['male_gold'].append('The '+i+',his')
        noise_male_bias['female_gold'].append('The '+i+',her')
    
    #PER PRON 
    new_sent = sent3.replace('MALE_BIAS', i).replace('MALE_NAME',female_name) #we put female name temporarily to have better masked predictions
    masked2_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}')
    alts = editor.suggest(masked2_sent,remove_duplicates = True)[:100]
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        print(group)
        print()
        male_sent = sent3.replace('MALE_BIAS', i).replace('MALE_NAME',male_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1])
        female_sent = sent4.replace('MALE_BIAS',i).replace('FEMALE_NAME',female_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1])
        noise_male_bias['male_referent'].append(male_sent)
        noise_male_bias['female_referent'].append(female_sent)
        noise_male_bias['anaphor_type'].append('PER_PRON')
        noise_male_bias['male_gold'].append('The '+i+',he')
        noise_male_bias['female_gold'].append('The '+i+',she')
    

In [ ]:
noise_male_bias_df = pd.DataFrame.from_dict(noise_male_bias)
bias = ['male']* len(noise_male_bias_df)
noise_male_bias_df['gender_bias']=bias
ambiguity = ['1'] * len(noise_male_bias_df)
noise_male_bias_df['ambiguity']=ambiguity
print(len(noise_male_bias_df))

In [ ]:
#case4
sent1 = case4[0][0] #male referent  POS PRON
sent2 = case4[0][1] #female referent  POS PRON
sent3 = case4[1][0] #male referent PER PRON
sent4 = case4[1][1] #female referent PER PRON
for i, female_name, male_name in zip(female_bias, female_names, male_names):
    print(female_name)
    print(male_name)
    #POS PRON
    new_sent = sent2.replace('FEMALE_BIAS', i).replace('FEMALE_NAME',male_name) #we put male name temporarily to have better masked predictions
    masked_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}').replace('ALIAS3','{mask}')
    alts = editor.suggest(masked_sent,remove_duplicates = True)[:100]
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        print(group)
        male_sent = sent1.replace('FEMALE_BIAS', i).replace('MALE_NAME',male_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        female_sent = sent2.replace('FEMALE_BIAS',i).replace('FEMALE_NAME',female_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1]).replace('ALIAS3',group[2])
        noise_female_bias['male_referent'].append(male_sent)
        noise_female_bias['female_referent'].append(female_sent)
        noise_female_bias['anaphor_type'].append('POS_PRON')
        noise_female_bias['male_gold'].append('The '+i+',his')
        noise_female_bias['female_gold'].append('The '+i+',her')
    
    #PER PRON 
    new_sent = sent4.replace('FEMALE_BIAS', i).replace('FEMALE_NAME',male_name) #we put female name temporarily to have better masked predictions
    masked2_sent = new_sent.replace('ALIAS1','{mask}').replace('ALIAS2','{mask}')
    alts = editor.suggest(masked2_sent,remove_duplicates = True)[:100]
    for group in alts:
        doc = nlp(group[0])
        for token in doc:
            if token.pos_ != 'VERB':
                break
        alts.remove(group)
        continue
    random_nums = np.random.choice(len(alts),3)
    for num in random_nums:
        group = alts[num]
        print(group)
        male_sent = sent3.replace('FEMALE_BIAS', i).replace('MALE_NAME',male_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1])
        female_sent = sent4.replace('FEMALE_BIAS',i).replace('FEMALE_NAME',female_name).replace('ALIAS1',group[0]).replace('ALIAS2',group[1])
        noise_female_bias['male_referent'].append(male_sent)
        noise_female_bias['female_referent'].append(female_sent)
        noise_female_bias['anaphor_type'].append('PER_PRON')
        noise_female_bias['male_gold'].append('The '+i+',he')
        noise_female_bias['female_gold'].append('The '+i+',she')
    

In [ ]:
noise_female_bias_df = pd.DataFrame.from_dict(noise_female_bias)
bias = ['female']* len(noise_female_bias_df)
noise_female_bias_df['gender_bias']=bias
ambiguity = ['1'] * len(noise_female_bias_df)
noise_female_bias_df['ambiguity']=ambiguity
print(len(noise_female_bias_df))

In [ ]:
gender_bias_df = no_noise_male_bias_df.append(no_noise_female_bias_df)
gender_bias_df = gender_bias_df.append(noise_male_bias_df)
gender_bias_df = gender_bias_df.append(noise_female_bias_df)

path = '../data/gender_anaphortype_ambiguity.csv'

gender_bias_df.to_csv(path)

In [ ]:
g_df = pd.read_csv(path)
